In [2]:
pip install transformers

     |████████████████████████████████| 3.3 MB 12.8 MB/s 
     |████████████████████████████████| 3.3 MB 36.7 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 61 kB 516 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install torch

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("SkolkovoInstitute/russian_toxicity_classifier")

model = AutoModelForSequenceClassification.from_pretrained("SkolkovoInstitute/russian_toxicity_classifier")

Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [5]:
from sklearn.model_selection import train_test_split

#Loading labeled dataset and splitting 80/20 train/test

training = pd.read_csv('labeled_2ch_pikabu.csv')
train_data = training.comment
train_labels = training.toxic

first_training_data, first_test_data, first_training_labels, first_test_labels = train_test_split(train_data, train_labels, test_size = 0.2)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import pipeline

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)
classifier(['Сука','Зайчик'])

[{'label': 'toxic', 'score': 0.9934903383255005},
 {'label': 'neutral', 'score': 0.8907330632209778}]

In [ ]:
pip install vk

  Created wheel for vk: filename=vk-2.0.2-py3-none-any.whl size=8275 sha256=7dc6a8db4bc72eabf22367a6d66b731d6736bf1d0370100ba4e8708ce5e97448
  Stored in directory: /root/.cache/pip/wheels/7b/d0/2b/d07628ebced43577b8001d0c9d716f972059535bbaadb1eb9d
Successfully built vk


In [ ]:
import vk
token = 'fe028e1dfe028e1dfe028e1d8cfe7beb4effe02fe028e1d9f45d4e443411241a567316a'
session = vk.Session(access_token=token)  # VK API authorization
vk_api = vk.API(session)

In [ ]:
def get_comments_from_vk_group(owner_id):
    
    #getting the ids of 100 posts
    group_posts_info = vk_api.wall.get(owner_id = owner_id, v = 5.95, count = 100)
    group_post_ids = []
    group_posts_items = group_posts_info['items']
    for i in group_posts_items:
      group_post_ids.append(i['id'])

    #getting the comment from these posts
    group_comments = []
    for group_post_id in group_post_ids:
      group_comments_info = vk_api.wall.getComments(owner_id = owner_id, post_id = group_post_id, v = 5.95, count = 100)
      group_comments_items = group_comments_info['items']
      for i in group_comments_items:
        group_comments.append(i['text'])
    print(f'Загружено {len(group_comments)} комментариев')
    group_comments = pd.DataFrame(group_comments)
    return group_comments

In [ ]:
#Downloading the comments from Lentach group

lentach_comments = get_comments_from_vk_group(-29534144)

In [9]:
lentach_comments= pd.read_excel('first_try_lentach_predictions.xlsx')

#lentach_comments.iloc[:,2] = classifier(list(lentach_comments.iloc[:,1]))

lentach_comments = lentach_comments.iloc[:,1]
listed = lentach_comments.astype(str).values.tolist()
predictions = classifier(listed[:50])
predictions

[{'label': 'neutral', 'score': 0.9973631501197815},
 {'label': 'toxic', 'score': 0.8335203528404236},
 {'label': 'toxic', 'score': 0.6858346462249756},
 {'label': 'neutral', 'score': 0.9993963241577148},
 {'label': 'neutral', 'score': 0.9970263838768005},
 {'label': 'neutral', 'score': 0.961527943611145},
 {'label': 'neutral', 'score': 0.9617114663124084},
 {'label': 'neutral', 'score': 0.9912154674530029},
 {'label': 'neutral', 'score': 0.9475409984588623},
 {'label': 'neutral', 'score': 0.9406463503837585},
 {'label': 'neutral', 'score': 0.9994232654571533},
 {'label': 'neutral', 'score': 0.9740821123123169},
 {'label': 'neutral', 'score': 0.9980717897415161},
 {'label': 'neutral', 'score': 0.8315345644950867},
 {'label': 'neutral', 'score': 0.9691274166107178},
 {'label': 'neutral', 'score': 0.828262984752655},
 {'label': 'neutral', 'score': 0.9600170850753784},
 {'label': 'toxic', 'score': 0.9264360666275024},
 {'label': 'neutral', 'score': 0.9716460704803467},
 {'label': 'neutral'

In [24]:
predictions = pd.DataFrame(predictions)
predictions['comment'] = lentach_comments

In [25]:
predictions

,label,score,comment
0,neutral,0.997363,мой палец: *чуть влажный*\n\nсканер отпечатка ...
1,toxic,0.833520,"Ахренкть, адмен украл мой коммент к клмменту, ..."
2,toxic,0.685835,У топы был классный видос на эту тему
3,neutral,0.999396,чтобы не попасть в тюрьму носите перчатки и маски
4,neutral,0.997026,Первая два три четыре пять
5,neutral,0.961528,"Боже, да что не так с этим СНГ"
6,neutral,0.961711,Да ну нах... подождите. Только что такая новос...
7,neutral,0.991215,"Это точно не с Панорамы, а?"
8,neutral,0.947541,На экспонате лишняя деталь. Рожок с патронами.
9,neutral,0.940646,Деменция болезнь сложная. Главное вовремя нача...
